In [35]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# ฟังก์ชันคำนวณค่าต่างๆ สำหรับการประเมินผลข้อมูล
def data_validation(meas,esti):
    #meas เเทนค่าจริง
    #esti เเทนค่าที่ได้จากการคำนวณหรือโมเดล
    #normalized factors เป็นค่าเฉลี่ยของ ground truth
    mae = np.abs((meas - esti)).mean()
    cor = meas.corr(esti)
    rmse = np.sqrt(((meas - esti) ** 2).mean())
    nrmse = 100*rmse/meas.mean()
    mbe = (esti - meas).mean()
    num = len(meas)
    nmbe = 100*mbe/meas.mean()
    nmae = 100*mae/meas.mean()

    # คืนค่าต่างๆ ที่คำนวณได้
    return [mae,rmse,mbe,cor,num,nrmse,nmae,nmbe]

# ฟังก์ชันคำนวณค่าต่างๆ และจัดเก็บใน DataFrame
def calculate_metrics(meas,esti,Site_id,Station_name):

    # เรียกใช้ฟังก์ชัน data_validation เพื่อคำนวณค่าต่างๆ
    [mae,rmse,mbe,cor,num,nrmse,nmae,nmbe] = data_validation(meas,esti)
    
    # สร้าง DataFrame ที่เก็บค่าต่างๆที่คำนวณเเล้ว
    return pd.DataFrame([{
        'site_id': Site_id,
        'site_name': Station_name,
        'number of sample': num,
        'MAE': f'{mae:.3f}',
        'nMAE': f'{nmae:.3f}%',
        'RMSE': f'{rmse:.3f}',
        'nRMSE': f'{nrmse:.3f}%',
        'MBE': f'{mbe:.3f}',
        'nMBE': f'{nmbe:.3f}%',
        'correlation': f"{cor:.3f}"}])

In [ ]:
# อ่านข้อมูล metadata
site_df = pd.read_csv('../data/DeDe_49stations/DeDe_49sites_metadata.csv')

In [ ]:
# CAMS and ERA5 vs DeDe (15 min)
#สร้าง DataFrame สำหรับเก็บผลลัพธ์
table_cams = pd.DataFrame()
table_era5 = pd.DataFrame()

# สร้าง Matrix Score สำหรับ CAMS และ ERA5 เเบบ 15 นาที
for i, row in site_df.iterrows():
    # อ่านข้อมูล radiation 
    df = pd.read_csv(f'../merged_data/solar_radiation_15min/solar_radiation_15min_site{row['Site_id']:02d}_2023.csv', parse_dates=['Datetime'])

    # กรองข้อมูลที่ไม่จำเป็น
    df.drop(columns=['Short-wave irradiation'],inplace=True)
    df.dropna(inplace=True)

    # กรองข้อมูลที่มีค่า I >= 10
    df = df[df['I'] >= 10]

    # กำหนด meas
    meas = df['I']

    # เปลี่ยนหน่วยให้เป็น W/m^2 เเล้วกำหนดให้เป็นค่า esti สำหรับ CAMS
    esti = df['GHI']*4 

    # คำนวณค่าต่างๆ และจัดเก็บใน DataFrame ของ CAMS
    table_cams = pd.concat([table_cams, calculate_metrics(meas, esti, row['Site_id'], row['Station_name'])], ignore_index=True)

    # เปลี่ยนหน่วยให้เป็น W/m^2 เเล้วกำหนดให้เป็นค่า esti สำหรับ ERA5
    esti = df['ssrd']/3600

    # คำนวณค่าต่างๆ และจัดเก็บใน DataFrame ของ ERA5
    table_era5 = pd.concat([table_era5, calculate_metrics(meas, esti, row['Site_id'], row['Station_name'])], ignore_index=True)

# merge DataFrame ของ CAMS และ ERA5 ด้วย site_id, site_name และ number of sample เเล้วเติมท้ายชื่อคอลัมน์ด้วย _CAMS และ _ERA5 ตามลำดับ
table_ts_15min = table_cams.merge(table_era5, on=['site_id', 'site_name', 'number of sample'], suffixes=('_CAMS', '_ERA5'))

# เรียงลำดับคอลัมน์ใน DataFrame
table_ts_15min = table_ts_15min[['site_id', 'site_name', 'number of sample',
            'MAE_CAMS', 'MAE_ERA5','nMAE_CAMS', 'nMAE_ERA5',
            'RMSE_CAMS', 'RMSE_ERA5','nRMSE_CAMS', 'nRMSE_ERA5',
            'MBE_CAMS', 'MBE_ERA5','nMBE_CAMS', 'nMBE_ERA5',
            'correlation_CAMS', 'correlation_ERA5']]

In [ ]:
# CAMS and ERA5 vs DeDe (1 hr)
#สร้าง DataFrame สำหรับเก็บผลลัพธ์
table_cams = pd.DataFrame()
table_era5 = pd.DataFrame()

# สร้าง Matrix Score สำหรับ CAMS และ ERA5 เเบบ 1 ชั่วโมง
for i, row in site_df.iterrows():
    # อ่านข้อมูล radiation
    df = pd.read_csv(f'../merged_data/solar_radiation_1hr/solar_radiation_1hr_site{row['Site_id']:02d}_2023.csv', parse_dates=['Datetime'])

    # กรองข้อมูลที่ไม่จำเป็น
    df.drop(columns=['Short-wave irradiation'],inplace=True)
    df.dropna(inplace=True)

    # กรองข้อมูลที่มีค่า I >= 10
    df = df[df['I'] >= 10]

    # กำหนด meas
    meas = df['I']

    # เปลี่ยนหน่วยให้เป็น W/m^2 เเล้วกำหนดให้เป็นค่า esti สำหรับ CAMS
    esti = df['GHI']

    # คำนวณค่าต่างๆ และจัดเก็บใน DataFrame ของ CAMS
    table_cams = pd.concat([table_cams, calculate_metrics(meas, esti, row['Site_id'], row['Station_name'])], ignore_index=True)

    # เปลี่ยนหน่วยให้เป็น W/m^2 เเล้วกำหนดให้เป็นค่า esti สำหรับ ERA5
    esti = df['ssrd']/3600

    # คำนวณค่าต่างๆ และจัดเก็บใน DataFrame ของ ERA5
    table_era5 = pd.concat([table_era5, calculate_metrics(meas, esti, row['Site_id'], row['Station_name'])], ignore_index=True)

# merge DataFrame ของ CAMS และ ERA5 ด้วย site_id, site_name และ number of sample เเล้วเติมท้ายชื่อคอลัมน์ด้วย _CAMS และ _ERA5 ตามลำดับ
table_ts_1hr = table_cams.merge(table_era5, on=['site_id', 'site_name', 'number of sample'], suffixes=('_CAMS', '_ERA5'))

# เรียงลำดับคอลัมน์ใน DataFrame
table_ts_1hr = table_ts_1hr[['site_id', 'site_name', 'number of sample',
            'MAE_CAMS', 'MAE_ERA5','nMAE_CAMS', 'nMAE_ERA5',
            'RMSE_CAMS', 'RMSE_ERA5','nRMSE_CAMS', 'nRMSE_ERA5',
            'MBE_CAMS', 'MBE_ERA5','nMBE_CAMS', 'nMBE_ERA5',
            'correlation_CAMS', 'correlation_ERA5']]

In [ ]:
# cams_forecast vs DeDe (15 min)

#สร้าง DataFrame สำหรับเก็บผลลัพธ์
table_fc_15min = pd.DataFrame()

# สร้าง Matrix Score สำหรับ CAMS forecast เเบบ 15 นาที
for i, row in site_df.iterrows():
    # อ่านข้อมูล CAMS forecast
    df = pd.read_csv(f'../merged_data/cams_forecast_15min/cams_forecast_15min_site{row['Site_id']:02d}_2023.csv', parse_dates=['Datetime'])

    # กำหนด index เป็น Datetime
    df.set_index('Datetime',inplace=True)

    # ดรอปค่า NaN และกรองข้อมูลที่อยู่ในช่วงเวลาที่ต้องการ
    df.dropna(inplace=True)
    df = df.between_time('07:00', '17:00')

    #กำหนด meas และ esti
    meas = df['ssrd'];esti = df['I']

    # คำนวณค่าต่างๆ และจัดเก็บใน DataFrame ของ CAMS forecast
    table_fc_15min = pd.concat([table_fc_15min, calculate_metrics(meas, esti, row['Site_id'], row['Station_name'])], ignore_index=True)

In [ ]:
# cams_forecast vs DeDe (1 hr)

#สร้าง DataFrame สำหรับเก็บผลลัพธ์
table_fc_1hr = pd.DataFrame()

# สร้าง Matrix Score สำหรับ CAMS forecast เเบบ 1 ชั่วโมง
for i, row in site_df.iterrows():
    # อ่านข้อมูล CAMS forecast
    df = pd.read_csv(f'../merged_data/cams_forecast_1hr/cams_forecast_1hr_site{row['Site_id']:02d}_2023.csv', parse_dates=['Datetime'])

    # กำหนด index เป็น Datetime
    df.set_index('Datetime',inplace=True)

    # ดรอปค่า NaN และกรองข้อมูลที่อยู่ในช่วงเวลาที่ต้องการ
    df.dropna(inplace=True)
    df = df.between_time('07:00', '17:00')

    # กำหนด meas และ esti
    meas = df['ssrd'];esti = df['I']

    # คำนวณค่าต่างๆ และจัดเก็บใน DataFrame ของ CAMS forecast
    table_fc_1hr = pd.concat([table_fc_1hr, calculate_metrics(meas, esti, row['Site_id'], row['Station_name'])], ignore_index=True)

In [ ]:
# บันทึกผลลัพธ์ของ Era5 กับ CAMS เเบบ 15 นาที และ 1 ชั่วโมง ลงไฟล์ CSV
table_ts_15min.to_csv('table_ts_15min.csv',index=False)
table_ts_1hr.to_csv('table_ts_1hr.csv',index=False)

In [ ]:
# บันทึกผลลัพธ์ของ CAMS forecast เเบบ 15 นาที และ 1 ชั่วโมง ลงไฟล์ CSV
table_fc_15min.to_csv('table_fc_15min.csv',index=False)
table_fc_1hr.to_csv('table_fc_1hr.csv',index=False)